In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import svm 
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import collections
import os
import pandas as pd
import networkx as nx
import math
import re
from difflib import SequenceMatcher
from unidecode import unidecode 
# from scipy.spatial.distance import euclidean, cosine

In [8]:
import numpy as np
import networkx as nx
import re
from scipy.spatial.distance import euclidean, cosine
import xgboost as xgb
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [10]:
import pandas as pd
import numpy as np
import gensim
# from gensim.models.fasttext import FastText
from string import punctuation
from nltk.corpus import stopwords
import re
punct = punctuation+'«»—…“”*№–'
stops = stopwords.words('english')
from collections import Counter,defaultdict
# from gensim.models.fasttext import load_facebook_model
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')

In [11]:
# edge embbeding functions
def avg_sum(v1, v2):
    return (np.array(v1)+np.array(v2))/2
def mult(v1, v2):
    return np.array(v1)*np.array(v2)
def w_l1(v1, v2):
    return np.abs(np.array(v1)-np.array(v2))
def w_l2(v1, v2):
    return (np.array(v1)-np.array(v2))**2
def nw_l1(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(n1)]
    neig2 = [n for n in graph.neighbors(n2)]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return np.abs((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))
def nw_l2(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(n1)]
    neig2 = [n for n in graph.neighbors(n2)]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return ((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))**2
def unknown_key_pressed(emb1, emb2):
    print('Нажата неизвестная клавиша')

In [12]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
def eud_cos(v1, v2):
    eud = euclidean_distances(v1, v2)
    cos = cosine_distances(v1, v2)
    return np.array((eud[0][0],cos[0][0]))

In [13]:
operators_4 = ['avg_sum', 'mult', 'w_l1', 'w_l2']
operators_2 = ['nw_l1', 'nw_l2']
# cosine cosine= {1: avg_sum, 2: mult, 3: w_l1, 4: w_l2}
# operators_2 = {1: nw_l1, 2: nw_l2}

In [19]:
# data load
Graph_train = nx.read_weighted_edgelist("train_test/edge_list_binary_train_2018.txt", delimiter=' ',nodetype=int)
Graph_test_2019 = nx.read_weighted_edgelist("train_test/edges_2019_binary.txt", delimiter=' ',nodetype=int)

In [21]:
#загрузка ребер с негатив семплинг
f_train = open('train_test/edge_list_train_NegSamp.txt')
f_2019 = open('train_test/edge_list_2019_NegSamp.txt')

edges_train = []
edges_2019 = []

for line in f_train:
    edges_train.append(line.split(' '))
for line in f_2019:
    edges_2019.append(line.split(' '))
f_train.close()
f_2019.close()

for i in range(len(edges_train)):
    edges_train[i][2] = edges_train[i][2][:-1]

for i in range(len(edges_2019)):
    edges_2019[i][2] = edges_2019[i][2][:-1]

In [22]:
edges_train_dict = {(e[0], e[1]) : e[2] for e in edges_train}
edges_2019_dict = {(e[0], e[1]) : e[2] for e in edges_2019}

In [23]:
nodes_train = []
for e in edges_train:
    nodes_train.extend(e[:2]) # экстендим парами и убираем дубликаты вершин
nodes_train = set(nodes_train)

nodes_2019 = []
for e in edges_2019:
    nodes_2019.extend(e[:2])
nodes_2019 = set(nodes_2019)


In [24]:
def model_rf(X_train, y_train, X_test, y_test):
    
    # RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    
    
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [25]:
def model_gbc(X_train, y_train, X_test, y_test):
    
    # GradientBoosting
    params = {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01}
    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
                     
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [26]:
def model_svm(X_train, y_train, X_test, y_test):

    # SVC
#     y_train = label_binarize(y_train, classes=[0, 1, 2, 3, 4])
    svm_model = svm.SVC()
#     svm_model = svm.SVC(probability=True)
    svm_model.fit(X_train, y_train)
#     print(X_train, "X_train")
#     print(y_train, 'y_train')
                     
    y_pred_train = svm_model.predict(X_train)
    y_pred_test = svm_model.predict(X_test)
#     y_pred_train_proba = svm_model.predict_proba(X_train)
#     y_pred_test_proba = svm_model.predict_proba(X_test)
#     print(y_pred_train_proba[0])

#     print(y_pred_train, 'y_pred_train')
#     labels = [0,1]
    
    precision_train = precision_score(y_train, y_pred_train, average=None)
    precision_test = precision_score(y_test, y_pred_test, average=None)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [27]:
def model_xgb(X_train, y_train, X_test, y_test):
#     param = {
#    'max_depth': 3,
#    'eta': 0.3, 
#    'silent': 1, 
#    'objective': 'multi:softprob',
#    'num_class': 2}
#     num_round = 20
    param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
    num_round = 10
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
#     clf = xgboost.XGBClassifier()
    clf = xgb.train(param, dtrain, num_round)
#     clf = XGBRegressor()
#     clf.fit(X_train, y_train, verbose=False)
    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    clf = xgb.train(param, dtrain, num_round)
    
#     clf.fit(X_train, y_train)
#     predictions = [round(value) for value in y_pred]
    y_pred_train_prob = clf.predict(dtrain)
    y_pred_train = [int(round(value)) for value in y_pred_train_prob]
    y_pred_test_prob = clf.predict(dtest)
    y_pred_test = [int(round(value)) for value in y_pred_test_prob]
#     print(y_pred_test)
    
#     y_pred_train = clf.predict(X_train)
#     y_pred_train = np.asarray([np.argmax(line) for line in y_pred_train])
#     print(y_pred_train[0])
#     y_pred_test = clf.predict(X_test)
#     y_pred_test = np.asarray([np.argmax(line) for line in y_pred_test])

    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train_prob)
    logloss_test = log_loss(y_test, y_pred_test_prob)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  
    

In [28]:
# models = ['model_rf', 'model_xgb', 'model_gbc', 'model_svm']
functions = {1: model_rf, 2: model_xgb}
#3: model_gbc, 4: model_svm

In [39]:
for operator in range(len(operators_4)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    # f_emb_train = open('./Data/emb_done/out_2018.txt')
    f_emb = open('./Data/emb_done/out.txt')
    emb = []
    # emb_2018 = []
    for line in f_emb:
        emb.append(line.split(' '))
    # for line in f_emb_2018:
    #     emb_2018.append(line.split(' '))
    f_emb.close()
    # f_emb_2018.close()


    for i in range(len(emb)):
        emb[i][len(emb[i])-1] = emb[i][len(emb[i])-1][:-1] # для удаления /n на конце последнего числа эмбеддинга
    # for i in range(len(emb_2018)):
    #     emb_2018[i][len(emb_2018[i])-1] = emb_2018[i][len(emb_2018[i])-1][:-1]

    emb_dict = {} 
    for i in range(1,len(emb)):
        emb_dict[int(emb[i][0])] = [float(j) for j in emb[i][1:]] # собираем эмбеддинг в ключ-значение
    # emb_2018_dict = {} 
    # for i in range(1,len(emb_2018)):
    #     emb_2018_dict[int(emb_2018[i][0])] = [float(j) for j in emb_2018[i][1:]]

    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
    #         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
    #         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train.txt', 'a')
    results_test = open('results_test.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

In [42]:
for operator in range(len(operators_2)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    # f_emb_train = open('./Data/emb_done/out_2018.txt')
    f_emb = open('./Data/emb_done/out.txt')
    emb = []
    # emb_2018 = []
    for line in f_emb:
        emb.append(line.split(' '))
    # for line in f_emb_2018:
    #     emb_2018.append(line.split(' '))
    f_emb.close()
    # f_emb_2018.close()


    for i in range(len(emb)):
        emb[i][len(emb[i])-1] = emb[i][len(emb[i])-1][:-1] # для удаления /n на конце последнего числа эмбеддинга
    # for i in range(len(emb_2018)):
    #     emb_2018[i][len(emb_2018[i])-1] = emb_2018[i][len(emb_2018[i])-1][:-1]

    emb_dict = {} 
    for i in range(1,len(emb)):
        emb_dict[int(emb[i][0])] = [float(j) for j in emb[i][1:]] # собираем эмбеддинг в ключ-значение
    # emb_2018_dict = {} 
    # for i in range(1,len(emb_2018)):
    #     emb_2018_dict[int(emb_2018[i][0])] = [float(j) for j in emb_2018[i][1:]]

    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_test_2019,int(e[0]),int(e[1]),emb_dict)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_dict)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_2.txt', 'a')
    results_test = open('results_test_2.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

### LINE

In [ ]:

for operator in range(len(operators_4)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    # f_emb_train = open('./Data/emb_done/out_2018.txt')
    f_emb = open('./Data/emb_done/embedding_file_LINE.txt')
    emb = []
    # emb_2018 = []
    for line in f_emb:
        emb.append(line.split(' '))
    # for line in f_emb_2018:
    #     emb_2018.append(line.split(' '))
    f_emb.close()
    # f_emb_2018.close()


    for i in range(len(emb)):
        emb[i][len(emb[i])-1] = emb[i][len(emb[i])-1][:-1] # для удаления /n на конце последнего числа эмбеддинга
    # for i in range(len(emb_2018)):
    #     emb_2018[i][len(emb_2018[i])-1] = emb_2018[i][len(emb_2018[i])-1][:-1]

    emb_dict = {}
    for i in range(1,len(emb)):
        emb_dict[int(emb[i][0])] = [float(j) for j in emb[i][1:]] # собираем эмбеддинг в ключ-значение
    # emb_2018_dict = {} 
    # for i in range(1,len(emb_2018)):
    #     emb_2018_dict[int(emb_2018[i][0])] = [float(j) for j in emb_2018[i][1:]]

    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
    #         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emb_dict[int(e[0])]
            emb2 = emb_dict[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
    #         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_LINE.txt', 'a')
    results_test = open('results_test_LINE.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

### ABSTRACT

In [42]:
data = pd.read_csv('abstracts_keywords/node_attribute_abstract.csv')

In [43]:
def normalize(text):
    words = [word.strip(punct) for word in word_tokenize(text.lower())]
    text = ' '.join(words)
    text = re.sub(r'[\d+<>\&\$\^@#¯\(\)\{\}\[\]"©⊗→=\+μ∞×ℬℳℋπ]', '', text)
    text = re.sub(r'[\\\|\/;]', ' ', text) 
    return text

In [205]:
data['abstract_norm'] = data['abstract'].apply(lambda x: normalize(x))

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), norm='l2', stop_words=stops, max_features=10000,min_df=0.3, max_df=0.8)
vects_tfidf = tfidf.fit_transform(data['abstract_norm'])

In [211]:
vects_tfidf[0]

<1x1058468 sparse matrix of type '<class 'numpy.float64'>'
	with 397 stored elements in Compressed Sparse Row format>

In [213]:
emd_dict_abs_tf = dict(zip(data['node'],vects_tfidf))

In [45]:
vects_abst = np.load('abstracts_keywords/vects.npy')

In [46]:
vects_abst.shape

(46312, 300)

In [47]:
emd_dict_abs = dict(zip(data['node'],vects_abst))

In [53]:
for operator in range(len(operators_4)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emd_dict_abs[int(e[0])]
            emb2 = emd_dict_abs[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
    #         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emd_dict_abs[int(e[0])]
            emb2 = emd_dict_abs[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
    #         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_abs.txt', 'a')
    results_test = open('results_test_abs.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

In [52]:
for operator in range(len(operators_2)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emd_dict_abs[int(e[0])]
            emb2 = emd_dict_abs[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_test_2019,int(e[0]),int(e[1]),emd_dict_abs)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emd_dict_abs[int(e[0])]
            emb2 = emd_dict_abs[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emd_dict_abs)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_abs_2.txt', 'a')
    results_test = open('results_test_abs_2.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

In [214]:
# for operator in range(len(operators_4)+1)[1:]:
X_train = []
y_train = []
X_test = []
y_test = []
for e in edges_2019:
    if (e[0] in nodes_2019) and (e[1] in nodes_2019):
        emb1 = emd_dict_abs_tf[int(e[0])]
        emb2 = emd_dict_abs_tf[int(e[1])]
        w_19 = int(e[2])
        y_test.append(w_19)
#         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
#         res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
        res = eud_cos(emb1,emb2)
#         try:
#             w_train = float(edges_2018_dict[(e[0], e[1])])
#         except  KeyError :
#             w_train = 0
        #res = np.append(res,w_16)
        X_test.append(res)

for e in edges_train:
    if (e[0] in nodes_train) and (e[1] in nodes_train):
        emb1 = emd_dict_abs_tf[int(e[0])]
        emb2 = emd_dict_abs_tf[int(e[1])]
        w_train = int(e[2])
        y_train.append(w_train)
#         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
#         res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
        res = eud_cos(emb1, emb2)
#         try:
#             w_15 = float(edges_2015_dict[(e[0], e[1])])
#         except  KeyError :
#             w_15 = 0
        #res = np.append(res,w_15)
        X_train.append(res)
results_train = open('results_train_abs_tf.txt', 'a')
results_test = open('results_test_abs_tf.txt', 'a')
# results_train.write(str(operator)+'\n')
# results_test.write(str(operator)+'\n')
for keycode in range(len(functions)+1)[1:]:
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
    results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
    results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
results_train.close()
results_test.close()

KeyboardInterrupt: 

In [221]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

In [222]:
v1 = emd_dict_abs_tf[7004373913]
v2 = emd_dict_abs_tf[55177729700]

In [235]:
vector = np.array((eud[0][0],cos[0][0]))

In [243]:
vector

array([1.41118452, 0.99572087])

In [242]:
v2.shape

(1, 1058468)

In [229]:
cos = cosine_distances(v1,v2)

In [237]:
cos

array([[0.99572087]])

In [230]:
eud = euclidean_distances(v1,v2)

### KEYWORDS

In [54]:
data_kw = pd.read_csv('abstracts_keywords/node_attribute_keywords.csv')

In [32]:
data_kw = pd.read_csv('abstracts_keywords/node_attribute_keywords_PREP.csv')

In [33]:
data_kw.head()

,node,abstract,abstract_prep,abstract_to_vec
0,57021625100,Bioeconomic | Biotechnology | Environmental ec...,"['bioeconomic', 'biotechnology', 'environmenta...",bioeconomic biotechnology environmental econom...
1,7004373913,Infrare single-photon detectors | superconduct...,"['infrare single-photon detectors', 'supercond...",infrare single-photon detectors superconductin...
2,57202547372,Alcoho | Complexity | Heart rate variability |...,"['alcoho', 'complexity', 'heart rate variabili...",alcoho complexity heart rate variability syste...
3,55177729700,Coronar heart disease | Dietary inflammatory i...,"['coronar heart disease', 'dietary inflammator...",coronar heart disease dietary inflammatory ind...
4,27367833300,Boundarie | Prefrontal cortex | Sensorimotor l...,"['boundarie', 'prefrontal cortex', 'sensorimot...",boundarie prefrontal cortex sensorimotor learn...


In [119]:
data_kw.abstract[8]

'Antilocalizatio | Dephasing length | Electron transport | Ferromagnetic state | Magnetic properties | Topological insulator antiferromagnetis | ferromagnetism | magnetic Dipole-dipole interactions | magnetic impurities | platelet inclusions | RKKY interaction | topological insulators Antilocalizatio | Dephasing length | Electron transport | Ferromagnetic state | Magnetic properties | Topological insulator'

In [138]:
data_kw = data_kw.drop(columns=['abstract_prep'])

In [139]:
data_kw['abstract_prep']  = data_kw['abstract'].apply(lambda x: re.sub(r'^(\s\|\s)', '', x)).apply(lambda x: x.lower().split(' | '))

In [95]:
data_kw['abstract_prep'] = data_kw['abstract'].apply(lambda x: x.lower().split(' | '))

In [142]:
data_kw['abstract_prep'].iloc[8035]

['static analysis', 'strict aliasing', 'type-based alias analysis']

In [140]:
len(data_kw['abstract_prep'])

46312

In [99]:
len(vocab_list)

46312

In [143]:
vocab_list = list(data_kw['abstract_prep'])

In [144]:
vocab = listmerge3(vocab_list)

In [102]:
def listmerge3(lstlst):
    all=[]
    for lst in lstlst:
          all.extend(lst)
    return all

In [101]:
# [[word for word in line ]for line in vocab_list]

In [145]:
vocab_done = list(set(vocab))

In [181]:
data_kw.to_csv('abstracts_keywords/node_attribute_keywords_PREP.csv', index=False)

In [151]:
data_kw['abstract_to_vec'] = data_kw['abstract_prep'].apply(lambda x: ' '.join(x))

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), norm='l2', stop_words=stops, max_features=10000)
vects_tfidf = tfidf.fit_transform(data_kw['abstract_to_vec'])

In [147]:
# [[print(line) for word in line if word==''] for line in vocab_list]

In [159]:
X.shape

(46312, 59751)

In [176]:
np.array(X[0])

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [40]:
emd_dict_kw = dict(zip(data_kw['node'],vects_tfidf))

In [ ]:
# for operator in range(len(operators_4)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emd_dict_kw[int(e[0])]
            emb2 = emd_dict_kw[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
    #         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emd_dict_kw[int(e[0])]
            emb2 = emd_dict_kw[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
    #         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
            res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_kw.txt', 'a')
    results_test = open('results_test_kw.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()

In [ ]:
for operator in range(len(operators_2)+1)[1:]:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for e in edges_2019:
        if (e[0] in nodes_2019) and (e[1] in nodes_2019):
            emb1 = emd_dict_kw[int(e[0])]
            emb2 = emd_dict_kw[int(e[1])]
            w_19 = int(e[2])
            y_test.append(w_19)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_test_2019,int(e[0]),int(e[1]),emd_dict_kw)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_train = float(edges_2018_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_train = 0
            #res = np.append(res,w_16)
            X_test.append(res)

    for e in edges_train:
        if (e[0] in nodes_train) and (e[1] in nodes_train):
            emb1 = emd_dict_kw[int(e[0])]
            emb2 = emd_dict_kw[int(e[1])]
            w_train = int(e[2])
            y_train.append(w_train)
            res = operators_2.get(int(operator), unknown_key_pressed)(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emd_dict_kw)
#             res = operators_4.get(int(operator), unknown_key_pressed)(emb1, emb2)
    #         try:
    #             w_15 = float(edges_2015_dict[(e[0], e[1])])
    #         except  KeyError :
    #             w_15 = 0
            #res = np.append(res,w_15)
            X_train.append(res)
    results_train = open('results_train_kw_2.txt', 'a')
    results_test = open('results_test_kw_2.txt', 'a')
    results_train.write(str(operator)+'\n')
    results_test.write(str(operator)+'\n')
    for keycode in range(len(functions)+1)[1:]:
        precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = functions.get(int(keycode), unknown_key_pressed)(X_train, y_train, X_test, y_test)
        results_train.write(str(precision_train)+ ' ' + str(accuracy_train) + ' ' + str(f1_macro_train) + ' ' + str(f1_micro_train) + ' ' + str(logloss_train) + ' ' + str(roc_auc_train)+ '\n')
        results_test.write(str(precision_test)+ ' ' + str(accuracy_test) + ' ' + str(f1_macro_test) + ' ' + str(f1_micro_test) + ' ' + str(logloss_test) + ' ' + str(roc_auc_test)+ '\n')
    results_train.close()
    results_test.close()